In [1]:
import joblib

In [2]:
indexed_seq = joblib.load("./filter_indexed_repo.pkl")

In [3]:
indexed_seq[:3]

['|fragment|[3*]C(=O)C([4*])C|seed|[1*]C(=O)CCCCCCCCC(=O)C(=O)CCCCCCCCCCC(=O)CCCCC(C)C([1*])=O|rxn_smiles|[1*][*:1].[3*][*:2]>>[*:1][*:2] |atomProp:1.molAtomMapNumber.1:3.molAtomMapNumber.2:4.molAtomMapNumber.1:5.molAtomMapNumber.2||rxn_smarts|[1*]-[*:1].[3*]-[*:2]>>[$([C&D3]([#0,#6,#7,#8])=O):1]-&!@[$([O&D2]-&!@[#0,#6,#1]):2] |atomProp:1.molAtomMapNumber.1:3.molAtomMapNumber.2:4.molAtomMapNumber.1:5.molAtomMapNumber.2|',
 '|fragment|[4*]CC[4*]|seed|[3*]C(=O)CCCCCCCCCCC(=O)CCCC(C)N[5*]|rxn_smiles|[4*][*:1].[5*][*:2]>>[*:1][*:2] |atomProp:1.molAtomMapNumber.1:3.molAtomMapNumber.2:4.molAtomMapNumber.1:5.molAtomMapNumber.2||rxn_smarts|[4*]-[*:1].[5*]-[*:2]>>[$([C&!D1&!$(C=*)]-&!@[#6]):1]-&!@[$([N&!D1&!$(N=*)&!$(N-[!#6&!#16&!#0&!#1])&!$([N&R]@[C&R]=O)]):2] |atomProp:1.molAtomMapNumber.1:3.molAtomMapNumber.2:4.molAtomMapNumber.1:5.molAtomMapNumber.2|',
 '|fragment|[1*]C(=O)CCCCCCCCCCC([3*])=O|seed|[5*]NC(C)C(=O)CCc1ccc([16*])cc1|rxn_smiles|[16*][*:2].[3*][*:1]>>[*:1][*:2] |atomProp:1.molAto

In [4]:
len([x for x in indexed_seq if "[6*]" in x ])

304522

In [7]:
import numpy as np

mean_length = np.mean([len(x.split("|")[2] + x.split("|")[4]) for x in indexed_seq])

In [ ]:
max_length = 45

np.float64(50.431153260194606)

In [12]:
unique_seqs = set()

In [7]:
def parser(str_val):
    values = str_val.split("|")
    return values[2], values[4], values[10]

In [13]:
prohibit_wildcards = ["6", "7", "8", "9", "11"]

In [15]:
allowed_extra_symbols = ["O", "=O"]

In [18]:
def calculate_max_continuous_C_sequence(sequence):
    max_count = 0
    current_count = 0
    for char in sequence:
        if char == 'C':
            current_count += 1
            max_count = max(max_count, current_count)
        else:
            current_count = 0
    return max_count

In [20]:
calculate_max_continuous_C_sequence("[4*]CC[4*]")

2

In [27]:
import tqdm

In [28]:
main_data = []
for i, seq in tqdm.tqdm(enumerate(indexed_seq)):
    frag, seed, smarts = parser(seq)
    if any([x in f"{frag}.{seed}" for x in prohibit_wildcards]):
        continue
    if not (any([x in frag for x in allowed_extra_symbols])
            and any([x in seed for x in allowed_extra_symbols])):
        # print(f"one of {seed} or {frag} does not have allowed extra symbols")
        continue
    max_C_sequence = calculate_max_continuous_C_sequence(f"{frag}.{seed}")
    main_data.append((f"{frag}.{seed}|{smarts}", max_C_sequence))

1573357it [00:09, 174610.98it/s]


In [29]:
len(main_data)

735006

In [30]:
main_data[:3]

[('[3*]C(=O)C([4*])C.[1*]C(=O)CCCCCCCCC(=O)C(=O)CCCCCCCCCCC(=O)CCCCC(C)C([1*])=O|[1*]-[*:1].[3*]-[*:2]>>[$([C&D3]([#0,#6,#7,#8])=O):1]-&!@[$([O&D2]-&!@[#0,#6,#1]):2] ',
  11),
 ('[1*]C(=O)C([3*])C.[4*]CCOCC(O)C(=O)C([4*])C|[3*]-[*:1].[4*]-[*:2]>>[$([O&D2]-&!@[#0,#6,#1]):1]-&!@[$([C&!D1&!$(C=*)]-&!@[#6]):2] ',
  2),
 ('[1*]C([3*])=O.[3*]CC(O)CCCC(=O)C(C)CCC[4*]|[1*]-[*:1].[3*]-[*:2]>>[$([C&D3]([#0,#6,#7,#8])=O):1]-&!@[$([O&D2]-&!@[#0,#6,#1]):2] ',
  4)]

In [ ]:
sorted(["somethink", "something"])

['something', 'somethink']

In [38]:
outros = ["|0.5|0.5|<1-3:0.5:0.5<1-4:0.5:0.5<2-3:0.5:0.5<2-4:0.5:0.5", "|0.5|0.5|<1-2:0.375:0.375<1-1:0.375:0.375<2-2:0.375:0.375<3-4:0.375:0.375<3-3:0.375:0.375<4-4:0.125:0.125<1-3:0.125:0.125<1-4:0.125:0.125<2-3:0.125:0.125<2-4:0.125:0.125"]

In [39]:
main_data_w_outro = []
for data in main_data:
    for outro in outros:
        main_data_w_outro.append((data[0] + outro, data[1]))

In [40]:
len(main_data_w_outro)

1470012

In [41]:
import pandas as pd
df = pd.DataFrame(main_data_w_outro, columns=["sequence", "max_C_sequence"])

In [42]:
df.head()

,sequence,max_C_sequence
0,[3*]C(=O)C([4*])C.[1*]C(=O)CCCCCCCCC(=O)C(=O)C...,11
1,[3*]C(=O)C([4*])C.[1*]C(=O)CCCCCCCCC(=O)C(=O)C...,11
2,[1*]C(=O)C([3*])C.[4*]CCOCC(O)C(=O)C([4*])C|[3...,2
3,[1*]C(=O)C([3*])C.[4*]CCOCC(O)C(=O)C([4*])C|[3...,2
4,[1*]C([3*])=O.[3*]CC(O)CCCC(=O)C(C)CCC[4*]|[1*...,4


In [43]:
# get unique sequences
unique_sequences = df["sequence"].unique()

In [44]:
unique_sequences.shape

(1470012,)

In [ ]:
df.to_csv("finals/dfs/filtered_sequences_case_study.csv", index=False)

In [46]:
sample_df = df.iloc[:1000, :]

In [47]:
sample_df.head()

,sequence,max_C_sequence
0,[3*]C(=O)C([4*])C.[1*]C(=O)CCCCCCCCC(=O)C(=O)C...,11
1,[3*]C(=O)C([4*])C.[1*]C(=O)CCCCCCCCC(=O)C(=O)C...,11
2,[1*]C(=O)C([3*])C.[4*]CCOCC(O)C(=O)C([4*])C|[3...,2
3,[1*]C(=O)C([3*])C.[4*]CCOCC(O)C(=O)C([4*])C|[3...,2
4,[1*]C([3*])=O.[3*]CC(O)CCCC(=O)C(C)CCC[4*]|[1*...,4


In [48]:
sample_df.to_csv("finals/dfs/filtered_sequences_case_study_sample_1K.csv", index=False)